## HCBB gryphons_goal_3

In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import time

%matplotlib inline

## 1. What procedures could be reasonably associated with each individual Provider Type? (There are 88 types of Providers.)
- Filters for Physician/Other Supplier --Do we need a "credentials of the provider" column?
    - Entity Type = I
    - Place of Service = F and/or O
- What is the normal ‘basket’ of procedures for each Provider Type?

## 2. Which procedure codes drive the most revenue for each Professional Provider Types?
- Exclude office visits

## Stretch: Create a process that predicts if an individual professional should be Provider Type ‘Orthopedic Surgery’ based on the procedure codes they have data for.
- How often does this data-driven expected Provider Type of ‘Orthopedic Surgery’ differ from the listed Provider Type for given professional?

## Import pkl data, from first project notebook

In [ ]:
providers_and_procedures = pd.read_pickle("../data/hcpcs_providers_clean.pkl")

## begin EDA of dataset

In [3]:
providers_and_procedures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 23 columns):
National Provider Identifier                                int64
Last Name/Organization Name of the Provider                 object
First Name of the Provider                                  object
Credentials of the Provider                                 object
Entity Type of the Provider                                 object
Street Address 1 of the Provider                            object
City of the Provider                                        object
Zip Code of the Provider                                    object
State Code of the Provider                                  object
Country Code of the Provider                                object
Provider Type                                               object
Medicare Participation Indicator                            object
Place of Service                                            object
HCPCS Code       

In [4]:
providers_and_procedures.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Credentials of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400
1,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154
2,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385
3,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763
4,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000


In [5]:
providers_and_procedures['Place of Service'].value_counts()

O    5740315
F    3675810
Name: Place of Service, dtype: int64

In [14]:
providers_and_procedures['Provider Type'].unique()

array(['Internal Medicine', 'Pathology', 'Anesthesiology',
       'Family Practice', 'Obstetrics & Gynecology', 'General Surgery',
       'Urology', 'Cardiac Surgery', 'Physician Assistant',
       'Physical Therapist in Private Practice', 'Dermatology',
       'Licensed Clinical Social Worker', 'Optometry',
       'Physical Medicine and Rehabilitation', 'Radiation Oncology',
       'Infectious Disease', 'Orthopedic Surgery',
       'Certified Clinical Nurse Specialist', 'Endocrinology',
       'Chiropractic', 'Psychiatry', 'Diagnostic Radiology',
       'Rheumatology', 'General Practice', 'Neurology',
       'Emergency Medicine', 'Nephrology', 'Ophthalmology',
       'Nurse Practitioner', 'Pulmonary Disease', 'Otolaryngology',
       'Plastic and Reconstructive Surgery', 'Psychologist, Clinical',
       'Cardiology', 'Interventional Cardiology', 'Gastroenterology',
       'Hospitalist', 'Audiologist',
       'Certified Registered Nurse Anesthetist (CRNA)',
       'Geriatric Medicine',

In [7]:
providers_and_procedures['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes    457835
Established patient office or other outpatient, visit typically 25 minutes    428262
New patient office or other outpatient visit, typically 45 minutes            179569
New patient office or other outpatient visit, typically 30 minutes            178057
Subsequent hospital inpatient care, typically 25 minutes per day              175335
                                                                               ...  
Revision of attachment of stomach to small bowel                                   1
Anesthesia for X-ray procedure on liver vein                                       1
Repair of non-healed foot bone                                                     1
Incision of ankle joint capsule to correct foot deformity                          1
Shoulder procedure                                                                 1
Name: HCPCS Description, Length: 5171, dtype: int64

## Begin work on Question 1. 
- create pivot table for provider types and HCPCS descriptions

In [84]:
providers_pivot_table = pd.pivot_table(providers_and_procedures, index = ['Provider Type', 'HCPCS Code', 
                                                                          'HCPCS Description', 'Place of Service'])

In [85]:
providers_pivot_table.head(75)

Average Medicare Allowed Amount  \
Provider Type      HCPCS Code HCPCS Description                                  Place of Service                                    
Addiction Medicine 36415      Insertion of needle into vein for collection of... O                                        3.000000   
                   36556      Insertion of central venous catheter for infusi... F                                      112.510000   
                   36620      Insertion of arterial catheter for blood sampli... F                                       47.990000   
                   70100      X-ray of mandible, less than 4 views               F                                        9.078696   
                                                                                 O                                       16.169743   
...                                                                                                                            ...   
                   90838      Psychotherapy, 60 minutes                          O                                       65.000000   
                   90847      Family psychotherapy including patient, 50 minutes O                                      102.760000   
                   90853      Group psychotherapy                                F                                       24.400000   
                                                                                 O                                       24.477547   
                   90870      Shock treatment and monitoring                     F                                      111.170000   

                                                                                                   Average Medicare Payment Amount  \
Provider Type      HCPCS Code HCPCS Description                                  Place of Service                                    
Addiction Medicine 36415      Insertion of needle into vein for collection of... O                                        2.933529   
                   36556      Insertion of central venous catheter for infusi... F                                       88.210000   
                   36620      Insertion of arterial catheter for blood sampli... F                                       37.620000   
                   70100      X-ray of mandible, less than 4 views               F                                        6.833478   
                                                                                 O                                       12.231949   
...                                                                                                                            ...   
                   90838      Psychotherapy, 60 minutes                          O                                       50.960000   
                   90847      Family psychotherapy including patient, 50 minutes O                                       75.680820   
                   90853      Group psychotherapy                                F                                       18.149977   
                                                                                 O                                       18.148445   
                   90870      Shock treatment and monitoring                     F                                       84.426744   

                                                                                                   Average Medicare Standardized Amount  \
Provider Type      HCPCS Code HCPCS Description                                  Place of Service                                         
Addiction Medicine 36415      Insertion of needle into vein for collection of... O                                             2.940000   
                   36556      Insertion of central venous catheter for infusi... F                                            97.910000   
                   36620      Insertion of arterial cathe

In [86]:
providers_pivot_table.shape

(56209, 8)

## Attempt subset of one Provider Type - 'Addiction Medicine'

In [67]:
addict_med_type = providers_and_procedures.loc[providers_and_procedures['Provider Type'] == 'Addiction Medicine']

In [68]:
addict_med_type.shape

(803, 23)

In [69]:
addict_med_type.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Credentials of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
40628,1003815515,BLANKENSHIP,KEVIN,MD,I,63 WHARF ST,MORGANTOWN,265015937,WV,US,...,99213,Established patient office or other outpatient...,N,99.0,11,99,69.06,125.000000,48.999798,52.045051
93826,1003998329,JONES,DOUGLAS,M.D.,I,6005 PARK AVENUE,MEMPHIS,381195213,TN,US,...,99231,"Subsequent hospital inpatient care, typically ...",N,15.0,12,15,37.01,135.000000,28.440000,31.230000
93827,1003998329,JONES,DOUGLAS,M.D.,I,6005 PARK AVENUE,MEMPHIS,381195213,TN,US,...,99232,"Subsequent hospital inpatient care, typically ...",N,73.0,27,73,68.28,150.342466,52.460000,57.390000
93828,1003998329,JONES,DOUGLAS,M.D.,I,6005 PARK AVENUE,MEMPHIS,381195213,TN,US,...,99233,"Subsequent hospital inpatient care, typically ...",N,241.0,49,241,98.57,175.000000,74.473071,83.354232
105632,1013072156,HORTON,TERRY,MD,I,501 W 14TH ST,WILMINGTON,198011013,DE,US,...,99222,"Initial hospital inpatient care, typically 50 ...",N,11.0,11,11,141.53,251.894545,110.960000,108.890000


In [70]:
addiction_pivot_table = pd.pivot_table(addict_med_type, index = ['Provider Type', 'HCPCS Description'])

In [73]:
addiction_pivot_table.shape

(139, 8)

## Turn full pivot table into a dataframe

In [87]:
## df = table.reset_index() 
prov_desc_df = providers_pivot_table.reset_index()

In [88]:
prov_desc_df

,Provider Type,HCPCS Code,HCPCS Description,Place of Service,Average Medicare Allowed Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Average Submitted Charge Amount,National Provider Identifier,Number of Distinct Medicare Beneficiary/Per Day Services,Number of Medicare Beneficiaries,Number of Services
0,Addiction Medicine,36415,Insertion of needle into vein for collection o...,O,3.000000,2.933529,2.940000,14.714286,1.443171e+09,85.000000,52.285714,85.000000
1,Addiction Medicine,36556,Insertion of central venous catheter for infus...,F,112.510000,88.210000,97.910000,129.390000,1.770876e+09,11.000000,11.000000,12.000000
2,Addiction Medicine,36620,Insertion of arterial catheter for blood sampl...,F,47.990000,37.620000,41.360000,55.190000,1.770876e+09,13.000000,12.000000,13.000000
3,Addiction Medicine,70100,"X-ray of mandible, less than 4 views",F,9.078696,6.833478,7.167391,35.000000,1.306066e+09,46.000000,39.000000,46.000000
4,Addiction Medicine,70100,"X-ray of mandible, less than 4 views",O,16.169743,12.231949,13.277022,44.816176,1.306066e+09,541.000000,471.000000,544.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
56204,Vascular Surgery,Q4131,"Epifix or epicord, per square centimeter",O,161.975351,126.948602,126.948611,446.500000,1.543785e+09,237.500000,27.000000,5138.000000
56205,Vascular Surgery,Q4148,"Neox 1k, per square centimeter",O,250.330000,196.258910,196.259052,588.000000,1.275593e+09,27.000000,11.000000,211.000000
56206,Vascular Surgery,Q9965,"Low osmolar contrast material, 100-199 mg/ml i...",O,0.834526,0.654269,0.654271,2.000000,1.609892e+09,111.000000,70.000000,5786.000000
56207,Vascular Surgery,Q9966,"Low osmolar contrast material, 200-299 mg/ml i...",O,0.246482,0.192448,0.193369,2.318315,1.531503e+09,93.891892,65.135135,5398.270270


In [89]:
prov_desc_df['HCPCS Description'].value_counts()

Vaccine for influenza for administration into muscle, 0.5 ml dosage           215
Ultrasound study of arteries of both arms and legs                            159
Injections of lower or sacral spine facet joint using imaging guidance        154
Established patient office or other outpatient visit, typically 15 minutes    144
Established patient office or other outpatient, visit typically 25 minutes    144
                                                                             ... 
Removal or biopsy of sacral spine bone growth                                   1
17-hydroxypregnenolone (hormone) level                                          1
Fusion of thumb at wrist with bone graft                                        1
Release of ear bone                                                             1
Repositioning or removal of external dialysis tube from vein to artery          1
Name: HCPCS Description, Length: 5171, dtype: int64

## Maybe use a dictionary to sort HCPCS descriptions by Provider Type?

In [110]:
# Initialize an empty dictionary: 

provider_type_dict ={} ## could also be an empty df

# Iterate over the file
    
    # Iterate over the column in DataFrame
for key, val in prov_desc_df.iterrows():
    if val['Provider Type'] not in provider_type_dict.keys():
        provider_type_dict[val['Provider Type']]=[val['HCPCS Description']] ##, val['Place of Service']]
        
    elif (val['HCPCS Description'] not in provider_type_dict[val['Provider Type']]):
        provider_type_dict[val['Provider Type']].append(val['HCPCS Description'])
        ##provider_type_dict[val['Provider Type']].append(val['Place of Service'])
    
    else:
        continue


# Print the populated dictionary
print(provider_type_dict)

{'Addiction Medicine': ['Insertion of needle into vein for collection of blood sample', 'Insertion of central venous catheter for infusion, patient 5 years or older', 'Insertion of arterial catheter for blood sampling or infusion, accessed through the skin', 'X-ray of mandible, less than 4 views', 'X-ray of teeth, less than full mouth', 'Imaging of voice box with speech evaluation', 'X-ray of chest, 2 views, front and side', 'X-ray of spine of neck, 2 or 3 views', 'X-ray of wrist, minimum of 3 views', 'X-ray of hand, minimum of 3 views', 'X-ray of hip with pelvis, 2-3 views', 'X-ray of knee, 3 views', 'X-ray of ankle, minimum of 3 views', 'X-ray of foot, minimum of 3 views', 'X-ray of upper digestive tract', 'Blood test, comprehensive group of blood chemicals', 'Blood test, lipids (cholesterol and triglycerides)', 'Liver function blood test panel', 'Testing for presence of drug', 'Urinalysis, manual test', 'Automated urinalysis test', 'Analysis of urine', 'Urine pregnancy test', 'Stool

## Turn provider type/descriptions dictionary into a dataframe

In [115]:
provider_type_subset = pd.DataFrame(list(provider_type_dict.items()),columns = ['provider_type','hcpcs_description'])

In [116]:
provider_type_subset

,provider_type,hcpcs_description
0,Addiction Medicine,[Insertion of needle into vein for collection ...
1,Advanced Heart Failure and Transplant Cardiology,[Insertion of needle into vein for collection ...
2,All Other Suppliers,[Vaccine for influenza for injection into musc...
3,Allergy/ Immunology,"[Anesthesia for procedure on skin of arms, leg..."
4,Ambulance Service Provider,"[Ground mileage, per statute mile, Ambulance s..."
...,...,...
83,Thoracic Surgery,[Anesthesia for procedure on gastrointestinal ...
84,Undefined Physician type,[External EKG recording for more than 48 hours...
85,Unknown Supplier/Provider Specialty,[Insertion of central venous catheter for infu...
86,Urology,[Injection of biodegradable material adjacent ...


## There is a discrepancy between number of codes and the descriptions...

In [92]:
prov_desc_df['HCPCS Description'].nunique()

5171

In [93]:
prov_desc_df['HCPCS Code'].nunique()

5702

## Created a csv file for export to Tableau to further analysis

In [90]:
prov_desc_df.to_csv('prov_type_serv_desc.csv', index=False)

## Attempt to use plot(s) to visualize some parts of the data

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.bar(prov_desc_df['Provider Type'], prov_desc_df['Place of Service'])
plt.xlabel('Type')
plt.ylabel('Place of Service')
##plt.legend()
##  .set_xticklabels(years)
plt.title('Provider Types by Location', color = 'black');